In [3]:
import json
import pandas as pd
import itertools
import sys
import re
import string
import nltk
from collections import defaultdict
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [4]:
productFeatures_df = pd.read_csv("product_features.csv")
productFeatures_df.head()

,brand,description,gender,modelId,price,productName,rating,specs
0,Adidas,mi adizero adios 3 Shoes feature an energy-ret...,W,4002178_W,160,mi adizero adios 3 Shoes,1.8,Runner type: neut boost™ is our most responsiv...
1,Adidas,NaN,W,4002179_W,100,mi Energy Cloud Shoes,3.3,NaN
2,Adidas,"Designed specifically for women, these shoes a...",W,4002255_W,150,mi PureBoost X Shoes,5.0,Select single-color mesh or graphic-print mesh...
3,Adidas,Create your own adidas Supernova with boost™ t...,W,4002435_W,150,mi Supernova Shoes,3.9,Choose your outsole color for a custom footpri...
4,Adidas,The boost™ midsole in these running shoes rele...,M,4002556_M,160,mi PureBoost Shoes,4.5,boost™ is our most responsive cushioning ever:...


In [10]:
spec_text = productFeatures_df.loc[:, 'specs'].tolist()

In [21]:
type(spec_text[2])

str